# 🚀 OpenAI Python SDK 101

In this notebook we’ll learn how to interact with Large Language Models (LLMs) directly using the **OpenAI Python SDK**.  
This is the **first time** we’re exploring API interactions, so we’ll build up gradually:

1. **Initialize** the client with your API key.  
2. **Minimal call** to the API (Responses API).  
3. Use **Chat Completions** for system + user roles.  
4. Explore **temperature** (randomness) and **top_p** (nucleus sampling).  
5. Add **system prompts** to guide behavior.  
6. Try **streaming tokens** (like ChatGPT typing).  
7. Get **JSON/structured outputs** with schemas.  
8. Handle **errors, timeouts, and retries** gracefully.

By the end, you’ll know how to **call an LLM safely and flexibly** using just the OpenAI SDK.

In [54]:
import os, json, time
from typing import Any, Dict

### API key
- Set your OpenAI API key as an environment variable:  
  `export OPENAI_API_KEY="sk-..."` (macOS/Linux) or `setx OPENAI_API_KEY "sk-..."` (Windows, new terminal required).  
- In Colab: use `os.environ["OPENAI_API_KEY"] = "..."` (for demos only).


In [55]:
# OpenAI Python SDK v1 style
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [56]:
# Minimal "Responses API" call (recommended by OpenAI for new projects)
# Docs: https://platform.openai.com/docs/guides/text  and Responses vs Chat Completions
resp = client.responses.create(
    model="gpt-4o-mini",  # choose any available text-capable model
    input="In one sentence, explain the difference between temperature and top_p for sampling."
)
print(resp.output_text)

Temperature controls the randomness of predictions by scaling logit probabilities, while top-p (nucleus) sampling limits choices to the most probable tokens that cumulatively meet a specified probability threshold.


In [57]:
resp

Response(id='resp_07056a317840d2d30068f32ef5b3e08193b283c91c1e3b5838', created_at=1760767733.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_07056a317840d2d30068f32ef72650819382acc98e20ea9f6f', content=[ResponseOutputText(annotations=[], text='Temperature controls the randomness of predictions by scaling logit probabilities, while top-p (nucleus) sampling limits choices to the most probable tokens that cumulatively meet a specified probability threshold.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tie

In [58]:
# Using Chat Completions (still widely used & supported)
chat = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a concise teaching assistant."},
        {"role": "user", "content": "Give me 3 bullet points about overfitting."}
    ]
)
print(chat.choices[0].message.content)

- **Definition**: Overfitting occurs when a model learns the training data too well, capturing noise and outliers instead of the underlying patterns, resulting in poor generalization to new, unseen data.

- **Symptoms**: Indicators of overfitting include a significantly lower training error compared to validation or test error, and high model complexity relative to the amount of training data.

- **Prevention Strategies**: Techniques to prevent overfitting include using simpler models, implementing regularization methods, employing cross-validation, and using techniques like dropout in neural networks.


In [59]:
chat

ChatCompletion(id='chatcmpl-CRuPY2Egt90G5YE11SGTmhm3A2Hvd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- **Definition**: Overfitting occurs when a model learns the training data too well, capturing noise and outliers instead of the underlying patterns, resulting in poor generalization to new, unseen data.\n\n- **Symptoms**: Indicators of overfitting include a significantly lower training error compared to validation or test error, and high model complexity relative to the amount of training data.\n\n- **Prevention Strategies**: Techniques to prevent overfitting include using simpler models, implementing regularization methods, employing cross-validation, and using techniques like dropout in neural networks.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760767736, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='f

In [60]:
messages = [
    {"role": "system", "content": "You are a Python tutor who answers with short code examples."},
    {"role": "user", "content": "Show how to reverse a string in Python."}
]
r = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=2) #High temperature for more creative output
print(r.choices[0].message.content)

You can use slicing to reverse a string in Python. Here's an example:

```python
# Input string
original_string = "hello, world!"

# Reversed string
reversed_string = original_string[::-1]

print(reversed_string)  # Output: !dlrow ,olleh
```

This code creates a new string that is the reverse of `original_string` by utilizing slice notation.


In [61]:
from sys import stdout

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Write a short story about a cat and a dog."}],
    temperature=1,
    stream=True,
)

for event in stream:
    if hasattr(event, "choices"):
        delta = event.choices[0].delta
        if delta and delta.content:
            stdout.write(delta.content)
stdout.write("\n")

Once upon a time in a charming little village, there lived a cat named Whiskers and a dog named Bruno. Whiskers was a sleek gray tabby with emerald green eyes, known for her grace and cunning. Bruno, on the other hand, was a large, friendly golden retriever with a heart as big as his paws.

Despite their differences, the two animals enjoyed a curious friendship. They lived next door to each other; Whiskers claimed the warm windowsill of her owner's home, while Bruno basked in the sun on his porch. Every day, they would meet at the split fence that separated their yards, exchanging stories and planning their next adventure.

One sunny afternoon, as the birds chirped merrily, Whiskers proposed a daring idea. “Let’s go on an exploration to the old mill at the edge of the woods! I’ve heard tales of treasures hidden there,” she said, her eyes sparkling with excitement.

Bruno’s tail wagged enthusiastically. “I’m in! How do we get there?”

With a flick of her tail, Whiskers devised a route t

1

In [64]:
message = """I have bought 3 kg of Rice, 4 kg of Dhal, 3 packets of Biscuits, 2 kg of Suger, Format this as a list of json objects with each JSON object in the format:
{
"item": "<item name>", 
"quantity": <quantity in kg or packets> 
}

DO NOT include anything else in the response"""


completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    temperature=1,
    messages=[
        {"role": "user", "content": message}
    ],
)

print(completion.choices[0].message.content)

[
    {
        "item": "Rice",
        "quantity": 3
    },
    {
        "item": "Dhal",
        "quantity": 4
    },
    {
        "item": "Biscuits",
        "quantity": 3
    },
    {
        "item": "Sugar",
        "quantity": 2
    }
]


In [65]:
import json

response = completion.choices[0].message.content

items = json.loads(response)
type(items)


list

In [ ]:
for item in items:
    print(item["item"])

Rice
Dhal
Biscuits
Sugar


In [ ]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI


class Summary(BaseModel):
    topic: str
    key_points: List[str]


client = OpenAI()

completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    temperature=0,
    response_format=Summary,
    messages=[
        {"role": "user", "content": "Topic: Transformers in NLP. Give 3 key points."}
    ],
)

parsed = completion.choices[0].message.parsed
parsed

Summary(topic='Transformers in NLP', key_points=['Transformers utilize self-attention mechanisms to weigh the importance of different words in a sentence, allowing for better context understanding.', 'They enable parallel processing of data, significantly speeding up training times compared to traditional sequential models like RNNs.', 'Transformers have led to the development of powerful pre-trained models (e.g., BERT, GPT) that can be fine-tuned for various NLP tasks, achieving state-of-the-art results.'])